In [ ]:
import os
import glob
import numpy as np
from pint.models import get_model
import pint.logging
from astropy import units as u
from astropy.coordinates import SkyCoord
from astropy import constants as const

pint.logging.setup(level="WARNING")

In [ ]:
def deg_GBT(sep,cfreq,diam=100.0 * u.m):
    """ Calculate degradation given angular separation, center frequency
    
    This calculation assumes the beam's sensitivity pattern is well-approximated by a
    Gaussian function. The default diameter is for the GBT and the beam's full width at
    half max is calculated using FWHM = 1.2*lamda/eff_diam rad.
    
    Parameters
    ==========
    sep: quantity, separation between observed/actual positions (arcmin)
    cfreq: quantity, center frequency of observation (MHz)
    diam: optional quantity, effective diameter of telescope (meters)
    
    Returns
    =======
    deg: float, estimated degradation to sensitivity based on separation
    """
    lamda = const.c/cfreq
    fwhm = 1.2*(lamda/diam*u.rad).to(u.arcmin)
    sig = fwhm/2.35482 # (solve for sigma when amplitude is 1/2; see wiki)
    
    return np.exp(-(sep/sig)**2/2)

In [ ]:
MHz = 1.0e6*u.Hz
pars = np.sort(glob.glob("*[0-9]_swiggum+22.par")) # veto J1221 eclipse par

In [ ]:
data_dir = "./"

for p in pars:
    par_path = os.path.join(data_dir, p)
    mo = get_model(par_path)
    mfrd_path = os.path.join(data_dir,f"{mo.PSR.value}.mfrd")
    
    # mfrd files contain MJD, center frequency, RA, Dec info for each scan for a given pulsar
    with open(mfrd_path) as f:
        mfrd_lines = f.readlines()
    
    # For each scan, get the RA/Dec and compare to timing position (eqcoord)
    eqcoord = mo.coords_as_ICRS()
    
    fnames = []
    freqs = []
    separations = [] # for now, gather angular separations (arcmins)
    for ml in mfrd_lines:
        if ml.startswith('GUPPI'):
            fname, mjd_str, freq_str, ra_str, dec_str = ml.strip().split()
            c = SkyCoord(ra_str, dec_str, frame='icrs', unit=(u.hourangle, u.deg))
            fnames.append(fname)
            freqs.append(float(freq_str))
            separations.append(eqcoord.separation(c).arcminute)
    
    # Closer look at separations
    separations = np.array(separations)
    degradations = np.array([deg_GBT(s*u.arcmin,f*MHz) for s,f in zip(separations,freqs)])
    n820,n350 = freqs.count(820.0),freqs.count(350.0)
    print(f"PSR {mo.PSR.value} was observed {len(separations)} times in total")
    
    PRINT_FILES = False
    for ff in [350.0,820.0]:
        i_freq = np.where(np.array(freqs) == ff)[0]
        # separation thresholds (st) chosen to limit degradation to 10%
        if ff == 350.0:
            st = 7.0
            nt = n350
        if ff == 820.0:
            st = 3.0
            nt = n820
        nws = len(np.where(separations[i_freq]<st)[0]) # number within separation threshold
        print(
            f"  {nws} time(s) at {ff:.0f} MHz within {st:.0f} arcmin of the timing position ({nt} total)"
            )
        if separations[i_freq].any():
            minsep = np.min(separations[i_freq])
            mindeg = deg_GBT(minsep*u.arcmin,ff*MHz)
            medsep = np.median(separations[i_freq])
            meddeg = deg_GBT(medsep*u.arcmin,ff*MHz)
            print(f"  Min separation at {ff:.0f} MHz: {minsep:.1f} arcmin (deg = {mindeg:0.2f})")
            print(f"  Med separation at {ff:.0f} MHz: {medsep:.1f} arcmin (deg = {meddeg:0.2f})")
        else:
            print(f"  (No detections at {ff:.0f} MHz)")
            
        if PRINT_FILES:
            psr = p.split("_")[0]    
            sum_fname = f"{psr}_{int(ff)}.files"
            freq_cut = [(f==ff) for f in freqs]
            seps_cut = (separations < st)
            #print(np.array(fnames)[freq_cut*seps_cut])
            if nws != nt:
                with open(sum_fname,'w') as f:
                    for file in np.array(fnames)[freq_cut*seps_cut]:
                        f.write(file+'\n')